# 資料預處理

In [1]:
import numpy
from keras.datasets import cifar10
import numpy as np
import matplotlib.pyplot as plt
np.random.seed(10)
log_filepath  = './nin'

Using TensorFlow backend.


In [2]:
(X_img_train, y_label_train), (X_img_test, y_label_test) = cifar10.load_data()

In [3]:
print("train data:",'images:',X_img_train.shape," labels:",y_label_train.shape) 
print("test  data:",'images:',X_img_test.shape ," labels:",y_label_test.shape) 

train data: images: (50000, 32, 32, 3)  labels: (50000, 1)
test  data: images: (10000, 32, 32, 3)  labels: (10000, 1)


In [4]:
X_img_train_normalize = X_img_train / 255.0
X_img_test_normalize = X_img_test / 255.0

In [5]:
from keras.utils import np_utils
y_label_train_OneHot = np_utils.to_categorical(y_label_train)
y_label_test_OneHot = np_utils.to_categorical(y_label_test)

In [6]:
y_label_test_OneHot.shape

(10000, 10)

# 建立模型

In [7]:
from keras.callbacks import TensorBoard
tb_cb = TensorBoard(log_dir=log_filepath, histogram_freq=1)
cbks = [tb_cb]

/Users/lirt/anaconda/lib/python3.6/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [8]:
import keras 
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Input
from keras.layers import ZeroPadding2D
from keras.models import Model
from keras.layers import Dropout
from models import nin_cifar

In [ ]:
model=nin_cifar(input_shape=(32,32,3),classes=10)
model.compile(optimizer='sgd',loss='categorical_crossentropy',metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 16, 16, 192)       14592     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 160)       30880     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 96)        15456     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 8, 96)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 8, 8, 96)          0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 8, 8, 192)         460992    
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 8, 8, 192)         37056     
__________

# 訓練模型

In [ ]:
train_history=model.fit(X_img_train_normalize ,y_label_train_OneHot, epochs = 150,validation_split=0.2,verbose=1,batch_size=128,callbacks=cbks)

Train on 40000 samples, validate on 10000 samples
Epoch 1/150
40000/40000 [==============================] - 318s 8ms/step - loss: 2.4244 - acc: 0.0976 - val_loss: 2.4239 - val_acc: 0.1041
Epoch 2/150
40000/40000 [==============================] - 343s 9ms/step - loss: 2.4239 - acc: 0.1111 - val_loss: 2.4235 - val_acc: 0.1086
Epoch 3/150
40000/40000 [==============================] - 331s 8ms/step - loss: 2.4233 - acc: 0.1264 - val_loss: 2.4226 - val_acc: 0.1265
Epoch 4/150
40000/40000 [==============================] - 346s 9ms/step - loss: 2.4224 - acc: 0.1535 - val_loss: 2.4209 - val_acc: 0.1449
Epoch 5/150
40000/40000 [==============================] - 332s 8ms/step - loss: 2.4201 - acc: 0.1693 - val_loss: 2.4166 - val_acc: 0.1733
Epoch 6/150
40000/40000 [==============================] - 323s 8ms/step - loss: 2.4134 - acc: 0.1755 - val_loss: 2.4050 - val_acc: 0.1750
Epoch 7/150
40000/40000 [==============================] - 314s 8ms/step - loss: 2.3914 - acc: 0.1764 - val_loss: 2.

# 評估模型準確率

In [ ]:
#X_img_test_normalize
#y_label_test_OneHot
preds=model.evaluate(X_img_test_normalize,y_label_test_OneHot)
print("Validation loss="+str(preds[0]))
print("Validation accuracy="+str(preds[1]))

# 以圖形顯示訓練過程

In [ ]:
import matplotlib.pyplot as plt
def show_train_history(train_history,train,validation):
    plt.plot(train_history.history[train])
    plt.plot(train_history.history[validation])
    plt.title('Train History')
    plt.ylabel(train)
    plt.xlabel('Epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()

In [ ]:
show_train_history(train_history,'acc','val_acc')

In [ ]:
show_train_history(train_history,'loss','val_loss')

# 進行預測

In [ ]:
label_dict={0:"airplane",1:"automobile",2:"bird",3:"cat",4:"deer",
            5:"dog",6:"frog",7:"horse",8:"ship",9:"truck"}

In [ ]:
import matplotlib.pyplot as plt
def plot_images_labels_prediction(images,labels,prediction,
                                  idx,num=10):
    fig = plt.gcf()
    fig.set_size_inches(12, 14)
    if num>25: num=25 
    for i in range(0, num):
        ax=plt.subplot(5,5, 1+i)
        ax.imshow(images[idx],cmap='binary')
                
        title=str(i)+','+label_dict[labels[idx][0]]
        if len(prediction)>0:
            title+='=>'+label_dict[prediction[idx]]
            
        ax.set_title(title,fontsize=10) 
        ax.set_xticks([]);ax.set_yticks([])        
        idx+=1 
    plt.show()

In [ ]:
prediction=model.predict_classes(X_img_test_normalize)

In [ ]:
plot_images_labels_prediction(X_img_test,y_label_test,
                              prediction,0,10)

# confusion matrix

In [ ]:
import pandas as pd
pd.crosstab(y_label_test.reshape(-1),prediction,
            rownames=['label'],colnames=['predicat'])